In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')



import random

import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
class config:
    seed = 42
    device = "cuda:0"    
        
    lr = 1e-3
    epochs = 25
    batch_size = 32
    num_workers = 4
    train_5_folds = True

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(config.seed)

In [11]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [12]:
jittering = transforms.ColorJitter(brightness = (0.1,0.3), contrast =  (0.1,0.3), saturation = (0.1,0.4))


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [13]:

train1 = transforms.Compose([transforms.Resize((224,224)),
           transforms.ToTensor(),
           transforms.RandomHorizontalFlip(p=0.4),
          transforms.RandomVerticalFlip(p=0.4),
           transforms.RandomGrayscale(p=1),
           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train2 = transforms.Compose([transforms.Resize((224,224)),
           transforms.ToTensor(),
           transforms.RandomHorizontalFlip(p=0.4),
          transforms.RandomVerticalFlip(p=0.4),
           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
           jittering])

train3 = transforms.Compose([transforms.Resize((224,224)),
                             transforms.ToTensor(),
                                                 transforms.RandomHorizontalFlip(p=0.4),
                                                 transforms.RandomVerticalFlip(p=0.4),
                                                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                                 AddGaussianNoise(0., 0.05)])

train4 = transforms.Compose([transforms.Resize((224,224)),
                             transforms.ToTensor(),
                                                 transforms.RandomHorizontalFlip(p=0.5),
                                                 transforms.RandomVerticalFlip(p=0.5),
                                                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test = transforms.Compose([transforms.Resize((224,224)),
           transforms.ToTensor(),
           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])


In [14]:
trainset1 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=train1)
trainset2 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=train2)
trainset3 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=train3)

trainset4 = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=True,
                                        download=False,
                                        transform=train3)



vaildset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/share/cafir_10',
                                        train=False,
                                        download=False,
                                        transform=test)


trainset = trainset1 + trainset2 + trainset3 + trainset4



In [15]:
testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/share/images/Statistical_Deep_Image",
                                           transform = test)

In [16]:
train_loader = DataLoader(trainset,
                          batch_size = 32,
                          shuffle=True,
                          num_workers=2)

vaild_loader = DataLoader(vaildset,
                          batch_size = 64,
                          shuffle=True,
                          num_workers=2)


test_loader = DataLoader(testset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=2)

In [17]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

모델 넣기 

In [18]:
!pip install timm
import timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 13.8 MB/s 
     |████████████████████████████████| 182 kB 73.0 MB/s 


['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [19]:
net = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.0001)

total=0 
correct = 0

In [24]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [ ]:
for epoch in range(50):
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs= net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 750 ==749:
      print("Epoch: {},Batch : {}, Loss:{}".format(epoch+1, i+1, running_loss/2000))
      running_loss = 0.0
    if i %1500 ==1499:
      with torch.no_grad():
        val_loss = 0.0
        for k, data1 in enumerate(vaild_loader, 0):
          val_inputs, val_label = data1[0].to(device), data1[1].to(device)
          val_output = net(val_inputs)
          v_loss = criterion(val_output, val_label)
          val_loss += v_loss
          _, predicted = torch.max(val_output.data,1)
          total += val_label.size(0)
          correct += (predicted == val_label).sum().item()
      print("validation loss {}".format(val_loss))
      print("vaildset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0
      with torch.no_grad():
        for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs= net(images)
          _, predicted = torch.max(outputs.data,1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print("testset Accuracy  : {}".format(100* correct/total))
      total=0
      correct=0

Epoch: 1,Batch : 750, Loss:0.16700246343016625
Epoch: 1,Batch : 1500, Loss:0.15230648246780037
validation loss 81.05143737792969
vaildset Accuracy  : 82.55
testset Accuracy  : 19.25
Epoch: 1,Batch : 2250, Loss:0.1481673208884895
Epoch: 1,Batch : 3000, Loss:0.1437064092569053
validation loss 78.15939331054688
vaildset Accuracy  : 82.88
testset Accuracy  : 19.6


pretrained 넣기


In [ ]:
PATH = '/content/drive/MyDrive/share/trained_model/cifar_effnetv2_s.pth'



In [ ]:
torch.save(net.state_dict(), PATH)

In [ ]:
classes = ('plane','automobile','bird','cat','deer',
           'dog','frog','horse','ship','truck')

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    c= (predicted == labels).squeeze()
    for i in range(4):
      label= labels[i]
      class_correct[label]+= c[i].item()
      class_total[label]+= 1

for i in range(10):
  print("Accuracy of {} : {} %".format(classes[i],100* class_correct[i]/class_total[i]))

Accuracy of plane : 14.285714285714286 %
Accuracy of automobile : 8.333333333333334 %
Accuracy of bird : 8.333333333333334 %
Accuracy of cat : 8.333333333333334 %
Accuracy of deer : 10.714285714285714 %
Accuracy of dog : 12.5 %
Accuracy of frog : 20.833333333333332 %
Accuracy of horse : 8.333333333333334 %
Accuracy of ship : 10.714285714285714 %
Accuracy of truck : 16.666666666666668 %


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("trainset Accuracy  : {}".format(100* correct/total))

trainset Accuracy  : 96.398


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in vaild_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("vaildset Accuracy  : {}".format(100* correct/total))

vaildset Accuracy  : 69.06


In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_loader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs= net(images)
    _, predicted = torch.max(outputs.data,1)
    total +=labels.size(0)
    correct += (predicted == labels).sum().item()

print("testset Accuracy  : {}".format(100* correct/total))

testset Accuracy  : 18.35
